## 01. 패치화 outputsize 예시 이해하기 

https://pypi.org/project/patchify/

1.1. 2D

In [39]:
import numpy as np
from patchify import patchify, unpatchify

image = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print('이미지 사이즈 : ',image.shape)

patches = patchify(image, (2,2), step=1) # split image into 2*3 small 2*2 patches.
print('패치화된 사이즈 : ',patches.shape)

이미지 사이즈 :  (3, 4)
패치화된 사이즈 :  (2, 3, 2, 2)


In [32]:
## output size 계산
# patches.shape[0] = 2 = (3-2)/1 + 1 =2 # 잘려진 세로 개수 
# patches.shape[1] = 3 = (4-2)/1 + 1 =3 # 잘려진 가로 개수 
# patches.shape[2] = 2 # 필터 세로 사이즈 
# patches.shape[3] = 2 # 필터 가로 사이즈 

(2, 3, 2, 2)

In [40]:
image

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [41]:
patches

array([[[[ 1,  2],
         [ 5,  6]],

        [[ 2,  3],
         [ 6,  7]],

        [[ 3,  4],
         [ 7,  8]]],


       [[[ 5,  6],
         [ 9, 10]],

        [[ 6,  7],
         [10, 11]],

        [[ 7,  8],
         [11, 12]]]])

1.2. 3D

In [63]:
np.random.rand(2,3,3) # 3D 배열 이해 하기 

array([[[0.59030221, 0.58187063, 0.91775745],
        [0.99655855, 0.4557943 , 0.33534802],
        [0.05841365, 0.83957939, 0.87792055]],

       [[0.90287972, 0.97547266, 0.16740185],
        [0.55475702, 0.33588203, 0.28648255],
        [0.03578571, 0.59383602, 0.9610696 ]]])

In [107]:
import numpy as np
from patchify import patchify, unpatchify

image = np.random.rand(512,512,3)
print('이미지 사이즈 : ',image.shape)

patches = patchify(image, (2,2,3), step=2) # patch shape [2,2,3]
print('패치화된 사이즈 : ',patches.shape) # (52, 52, 1, 2, 2, 3). Total patches created: 511x511x1

이미지 사이즈 :  (512, 512, 3)
패치화된 사이즈 :  (256, 256, 1, 2, 2, 3)


In [108]:
reconstructed_image = unpatchify(patches, image.shape)

In [82]:
## output size 계산
# patches.shape[0] = 52 = (512-2)/10 + 1 =52 # 잘려진 세로 개수 
# patches.shape[1] = 52 = (512-2)/10 + 1 =52 # 잘려진 가로 개수 
# patches.shape[3] = 2 # 필터 세로 사이즈 
# patches.shape[4] = 2 # 필터 가로 사이즈 

In [97]:
 (512-2)/2 + 1

256.0

In [126]:
(512-5)%3

0

In [111]:
def patchify1(img, patch_shape):
    img = np.ascontiguousarray(img)  # won't make a copy if not needed
    X, Y = img.shape
    x, y = patch_shape
    shape = ((X-x+1), (Y-y+1), x, y) # number of patches, patch_shape
    # The right strides can be thought by:
    # 1) Thinking of `img` as a chunk of memory in C order
    # 2) Asking how many items through that chunk of memory are needed when indices
    #    i,j,k,l are incremented by one
    strides = img.itemsize*np.array([Y, 1, Y, 1])
    return np.lib.stride_tricks.as_strided(img, shape=shape, strides=strides)

In [112]:
def unpatchify1(img_patches, block_size):
    B0, B1 = block_size
    N = np.prod(img_patches.shape[1::2])
    patches2D = img_patches.transpose(0,2,1,3).reshape(-1,N)

    m,n = patches2D.shape
    row_mask = np.zeros(m,dtype=bool)
    col_mask = np.zeros(n,dtype=bool)
    row_mask[::B0]= 1
    col_mask[::B1]= 1
    row_mask[-B0:] = 1
    col_mask[-B1:] = 1
    return patches2D[np.ix_(row_mask, col_mask)]

In [113]:
img = np.random.randint(0,255,(16,25))
block_size = (4,8)

In [114]:
a= unpatchify1(patchify1(img, block_size), block_size)

In [121]:
patchify1(img, block_size).shape

(13, 18, 4, 8)

In [124]:
 (16-4)/1 + 1

13.0

In [105]:
np.allclose(img, unpatchify(patchify(img, block_size), block_size))

True